## CONVERSIÓN PDF EN CSV CON RECETAS

In [1]:
import pandas as pd
import pdfplumber
import re

def leer_pdf(ruta_archivo):
    texto_completo = ""
    try:
        with pdfplumber.open(ruta_archivo) as pdf:
            for pagina in pdf.pages:
                texto_completo += pagina.extract_text() + "\n"
        return texto_completo
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
        return None

def extraer_raciones(texto):
    patrones = [
        r'(\d+)\s*[Rr]aciones?',
        r'(\d+)/(\d+)\s*ración',
        r'(\d+)-(\d+)\s*[Rr]aciones?',
        r'[Pp]ara (\d+) personas?'  
    ]

    for patron in patrones:
        match = re.search(patron, texto)
        if match:
            if len(match.groups()) == 2:
                if '-' in texto:
                    return f"{match.group(1)}-{match.group(2)}"
                else:
                    return f"{match.group(1)}/{match.group(2)}"
            return match.group(1)
    return None

def procesar_receta(texto):
    lineas = texto.strip().split('\n')
    nombre = ""
    ingredientes = []
    pasos = []
    raciones = None
    
    paso_actual = ""
    en_paso = False

    for i, linea in enumerate(lineas):
        linea = linea.strip()

        if linea.startswith('[Receta]'):
            nombre = linea.replace('[Receta]', '').strip()
            # Buscar raciones en las siguientes 2 líneas
            for j in range(1, 3):
                if i + j < len(lineas):
                    rac = extraer_raciones(lineas[i + j])
                    if rac:
                        raciones = rac
                        break

        elif linea.startswith('·'):
            ingredientes.append(linea.replace('·', '').strip())

        elif re.match(r'^\d+\)', linea):
            # Detecta paso enumerado
            if paso_actual:
                pasos.append(paso_actual.strip())
            # Inicia nuevo paso
            paso_actual = linea.split(')', 1)[1].strip()
            en_paso= True
        
        elif en_paso and linea:
            paso_actual += " " + linea
    
    if paso_actual:
        pasos.append(paso_actual.strip())

    return {
        'nombre': nombre,
        'raciones': raciones,
        'ingredientes': ingredientes,
        'pasos': pasos
    }

# Leer el PDF
ruta_pdf = "Recetas text.pdf"  
texto_pdf = leer_pdf(ruta_pdf)

if texto_pdf:
    # Dividir el texto en recetas individuales
    recetas_texto = texto_pdf.split('[Receta]')
    recetas_texto = ['[Receta]' + r for r in recetas_texto[1:]]  # Excluir el primer elemento vacío

    # Procesar cada receta
    recetas_procesadas = []
    for receta in recetas_texto:
        datos_receta = procesar_receta(receta)
        recetas_procesadas.append(datos_receta)

    # Crear DataFrame
    df = pd.DataFrame(recetas_procesadas)

    # Mostrar el DataFrame
    print("\nPrimeras filas del DataFrame:")
    print(df.head())

    # Mostrar información sobre el DataFrame
    print("\nInformación del DataFrame:")
    print(df.info())


Primeras filas del DataFrame:
                                              nombre raciones  \
0  Huevos revueltos con de salmón, espinacas y qu...        2   
1                                    Rollitos de col     None   
2                             Fumet de gambas (ceto)        2   
3             Pudding de chía con arándanos y nueces        2   
4  Ensalada marinera: con aguacate, gambas, bacal...     None   

                                        ingredientes  \
0  [6 g de aceite de oliva (AOVE), 4 huevos, 20 g...   
1  [2 hojas de col (tiernas), 150 g de carne pica...   
2  [200 g de gambas peladas (ver observaciones), ...   
3  [250 ml leche de almendra, 250 ml de kefir (ca...   
4  [100 g Gambas, 100 g bacalao ahumado, ½ Aguaca...   

                                               pasos  
0  [Cortar las verduras: cebolla y espinacas y el...  
1  [Precalentar el horno a 180°C., Hervir 2 hojas...  
2  [Fundir la mantequilla en una olla grande a fu...  
3  [Una vez mezclado 

In [2]:
# Guardar en CSV
df.to_csv('recetas_con_raciones.csv', index=False)
print("\nArchivo CSV guardado como 'recetas_con_raciones.csv'")


Archivo CSV guardado como 'recetas_con_raciones.csv'


In [3]:
df.head()

,nombre,raciones,ingredientes,pasos
0,"Huevos revueltos con de salmón, espinacas y qu...",2,"[6 g de aceite de oliva (AOVE), 4 huevos, 20 g...",[Cortar las verduras: cebolla y espinacas y el...
1,Rollitos de col,None,"[2 hojas de col (tiernas), 150 g de carne pica...","[Precalentar el horno a 180°C., Hervir 2 hojas..."
2,Fumet de gambas (ceto),2,"[200 g de gambas peladas (ver observaciones), ...",[Fundir la mantequilla en una olla grande a fu...
3,Pudding de chía con arándanos y nueces,2,"[250 ml leche de almendra, 250 ml de kefir (ca...",[Una vez mezclado sirve en dos recipientes (un...
4,"Ensalada marinera: con aguacate, gambas, bacal...",None,"[100 g Gambas, 100 g bacalao ahumado, ½ Aguaca...","[Hierve las gambas, Cortar el aguacate y las h..."


## DF INGREDIENTES ESPECÍFICOS

In [4]:
df_ingredientes = df.explode("ingredientes")

In [5]:
df_ingredientes = df_ingredientes.groupby(["nombre", "ingredientes"]).sum()

In [6]:
df_ingredientes = df_ingredientes.drop(columns=["raciones", "pasos"])

In [7]:
# Vistazo a agrupación por ingredientes
df_ingredientes.head(50)

Empty DataFrame
Columns: []
Index: [(Ensalada marinera: con aguacate, gambas, bacalao ahumado y, 10 g Aceite de oliva), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, 100 g Gambas), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, 100 g bacalao ahumado), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, 20 g de nueces), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, 70 g Hojas verdes (lechuga, espinacas, rúcula, kale…)), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, Sal y pimienta), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, ½ Aguacate), (Ensalada marinera: con aguacate, gambas, bacalao ahumado y, ½ Limón), (Estofado de ternera con verduras, 1 cucharada de aceite de oliva), (Estofado de ternera con verduras, 1 cucharada de tomate concentrado (opcional)), (Estofado de ternera con verduras, 1 diente de ajo), (Estofado de ternera con verduras, 1 hoja de laurel), (Estofado de ternera con verduras, 1 taza (250 ml) de caldo de carne), (Estofado de ternera con verduras, 1 zanahoria), (Estofado de ternera con verduras, 1/2 cebolla), (Estofado de ternera con verduras, 100 g de judías verdes), (Estofado de ternera con verduras, 400 g de carne de ternera para estofar, troceada), (Estofado de ternera con verduras, Sal y pimienta al gusto), (Estofado de ternera con verduras, ½ cucharadita de tomín), (Fumet de gambas (ceto), 100 g de cebolla blanca), (Fumet de gambas (ceto), 15 g de cebollino o parte verde de una cebolleta), (Fumet de gambas (ceto), 2 dientes de ajo), (Fumet de gambas (ceto), 20 g de mantequilla), (Fumet de gambas (ceto), 200 g de gambas peladas (ver observaciones)), (Fumet de gambas (ceto), 400 g de coliflor), (Fumet de gambas (ceto), 500 ml de caldo de pollo o pescado), (Fumet de gambas (ceto), 60 ml de nata líquida), (Fumet de gambas (ceto), 6g de hojas de cilantro (opcional)), (Fumet de gambas (ceto), Sal al gusto), (Hamburguesa con ensalada, 10 g de aceite de oliva), (Hamburguesa con ensalada, 100g de ensalada verde), (Hamburguesa con ensalada, 200 g de carne de ternera del estofado del otro día), (Hamburguesa con ensalada, 200g de patata prebiótica), (Hamburguesa con ensalada, Sal y pimienta al gusto), (Hamburguesa con ensalada, ¼ de cucharadita de ajo en polvo o ½ ajo fresco), (Hamburguesa con ensalada, ¼ de cucharadita de cebolla en polvo), (Hamburguesa con ensalada, ¼ de cucharadita de mostaza Dijon), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 1 manojo de espinacas), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 100 g de salmón ahumado), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 20 g de cebolla), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 4 huevos), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 40 g de queso feta), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 50 g de queso crema), (Huevos revueltos con de salmón, espinacas y queso crema y feta, 6 g de aceite de oliva (AOVE)), (Merluza con corteza de almendras y parmesano + ensalada verde, 1 solomillo de merluza con piel (300g)), (Merluza con corteza de almendras y parmesano + ensalada verde, 1⁄2 limón), (Merluza con corteza de almendras y parmesano + ensalada verde, 20 g de queso parmesano), (Merluza con corteza de almendras y parmesano + ensalada verde, 25 g de almendras fileteadas), (Merluza con corteza de almendras y parmesano + ensalada verde, 25 g de mantequilla sin sal), (Merluza con corteza de almendras y parmesano + ensalada verde, Sal y pimienta al gusto)]

## DF INGREDIENTES + CANTIDAD + UNIDAD

In [8]:
#Convertimos agrupación en dataframe para obtener nuevas columnas (unidad, cantidad)
df_ingredientes = df_ingredientes.reset_index()

In [9]:
def process_ingredient(text):  
    if pd.isna(text):  
        return pd.Series({'ingrediente': None, 'cantidad': None, 'unidad': None})  

    text = str(text).strip()  

    # Patrones regex mejorados  
    patterns = [  
        # Patrón para "¼ de cucharadita de Ingrediente"  
        (r'^¼\s*(?:de\s+)?(cucharadita)\s+de\s+(.+)$', lambda m: (m.group(2), '1/4', m.group(1))),
        
        # Patrón para fracciones simples
        
        (r'^(?:1/4|1/2|1/3|¼|½|⅓)\s+(?:de\s+)?(.+)$', lambda m: (m.group(1), m.group(0).split()[0], None)),

        # Patrón para "X g/ml de Ingrediente"  
        (r'^(\d+)\s*(g)\s+(?:de\s+)?(.+)$', lambda m: (m.group(3), m.group(1), m.group(2))),  

        # Patrón para "X cucharada(s)/cucharadita(s) de Ingrediente"  
        (r'^(\d+)\s*(cucharada|cucharadita)s?\s+(?:de\s+)?(.+)$', lambda m: (m.group(3), m.group(1), m.group(2))),  

        # Patrón para "X taza (Y ml) de Ingrediente"  
        (r'^(\d+)\s*taza\s*$(\d+)\s*(ml)$\s*de\s*(.+)$', lambda m: (m.group(4), m.group(2), m.group(3))),  

        # Patrón para fracciones simples "1/2 Ingrediente" o "½ Ingrediente"  
        (r'^(?:1/2|½)\s+(.+)$', lambda m: (m.group(1), '1/2', None)),  

        # Patrón para "X diente(s) de ajo"  
        (r'^(\d+)\s*dientes?\s+de\s+(.+)$', lambda m: (m.group(2), m.group(1), 'unidad')),  

        # Patrón para fracciones con unidades "1/4 de cucharadita/cucharada de Ingrediente"  
        (r'^(?:1/4|¼)\s+(?:de\s+)?(cucharadita|cucharada)\s+de\s+(.+)$', lambda m: (m.group(2), '1/4', m.group(1))),
        
        # Patrón para "X ingrediente(s)"
        (r'^(\d+)\s+(.+)$', lambda m: (m.group(2), m.group(1), 'unidad')),
        
        # Patrón para "X ml" y "X ml de"
        (r'^(\d+)\s*ml(?:\s+de)?\s*(.+)$', lambda m: (m.group(2).strip(), m.group(1), 'ml')),
    ]  

    for pattern, handler in patterns:  
        match = re.match(pattern, text)  
        if match:  
            ingrediente, cantidad, unidad = handler(match)  
            return pd.Series({'ingrediente': ingrediente.strip(), 'cantidad': cantidad, 'unidad': unidad})  

    # Si no coincide con ningún patrón, devolver solo el ingrediente  
    return pd.Series({'ingrediente': text, 'cantidad': None, 'unidad': None})  

# Aplicar la función a cada ingrediente  
df_ingredientes[['ingrediente', 'cantidad', 'unidad']] = df_ingredientes['ingredientes'].apply(process_ingredient)  

# Limpiar y ajustar el resultado  
df_ingredientes['cantidad'] = df_ingredientes['cantidad'].replace('', None)  
df_ingredientes['unidad'] = df_ingredientes['unidad'].map({  
    'g': 'gramos',  
    'ml': 'mililitros',  
    'cucharada': 'cucharada',  
    'cucharadita': 'cucharadita',  
    'unidad': 'unidad'  
})  

# Limpiar paréntesis y texto opcional  
df_ingredientes['ingrediente'] = df_ingredientes['ingrediente'].apply(lambda x: re.sub(r'$[^)]*$', '', str(x)).strip())  
df_ingredientes['ingrediente'] = df_ingredientes['ingrediente'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())   

In [10]:
df_ingredientes.head(50)

,nombre,ingredientes,ingrediente,cantidad,unidad
0,"Ensalada marinera: con aguacate, gambas, bacal...",10 g Aceite de oliva,Aceite de oliva,10,gramos
1,"Ensalada marinera: con aguacate, gambas, bacal...",100 g Gambas,Gambas,100,gramos
2,"Ensalada marinera: con aguacate, gambas, bacal...",100 g bacalao ahumado,bacalao ahumado,100,gramos
3,"Ensalada marinera: con aguacate, gambas, bacal...",20 g de nueces,nueces,20,gramos
4,"Ensalada marinera: con aguacate, gambas, bacal...","70 g Hojas verdes (lechuga, espinacas, rúcula,...","Hojas verdes (lechuga, espinacas, rúcula, kale…)",70,gramos
5,"Ensalada marinera: con aguacate, gambas, bacal...",Sal y pimienta,Sal y pimienta,None,NaN
6,"Ensalada marinera: con aguacate, gambas, bacal...",½ Aguacate,Aguacate,½,NaN
7,"Ensalada marinera: con aguacate, gambas, bacal...",½ Limón,Limón,½,NaN
8,Estofado de ternera con verduras,1 cucharada de aceite de oliva,aceite de oliva,1,cucharada
9,Estofado de ternera con verduras,1 cucharada de tomate concentrado (opcional),tomate concentrado (opcional),1,cucharada


### INGREDIENTES + CANTIDAD + UNIDAD (Limpieza)

In [11]:
# Regex para limpiar "X ml + de + ingrediente"  

pattern = r'(\d+)\s*(ml)\s*de\s*(.*)'  

for idx, row in df_ingredientes.iterrows():  
    text = str(row["ingredientes"])   
    match = re.match(pattern, text)  

    if match:    
        cantidad = int(match.group(1))    
        ingrediente = match.group(3)  
 
        df_ingredientes.at[idx, "ingrediente"] = ingrediente  
        df_ingredientes.at[idx, "cantidad"] = cantidad  
        df_ingredientes.at[idx, "unidad"] = "mililitros"  

#### Kg en ingredientes (Limpieza caso aislado)

In [12]:
# Localizando Kg en ingredientes para manipularlo

df_ingredientes[df_ingredientes["ingredientes"].str.contains("kg", na=False)]

,nombre,ingredientes,ingrediente,cantidad,unidad
61,Pastel de pescado,1 kg de pescado limpio y desmenuzado (rap qued...,kg de pescado limpio y desmenuzado (rap queda ...,1,unidad


In [13]:
df_ingredientes.loc[61]["ingredientes"]

'1 kg de pescado limpio y desmenuzado (rap queda muy bien, sino también puedes'

In [14]:
# Cambiamos valores de forma manual al ser un caso aislado

df_ingredientes.loc[61, "ingrediente"] = "pescado limpio y desmenuzado"
df_ingredientes.loc[61, "unidad"] = "kilogramo"

In [15]:
df_ingredientes.loc[61]

nombre                                          Pastel de pescado
ingredientes    1 kg de pescado limpio y desmenuzado (rap qued...
ingrediente                          pescado limpio y desmenuzado
cantidad                                                        1
unidad                                                  kilogramo
Name: 61, dtype: object

#### ml leche de almendra (Limpieza caso aislado)

In [16]:
# localizamos el valor de ingrediente que no recoge regex

df_ingredientes[df_ingredientes["ingrediente"].str.contains("ml leche")]

,nombre,ingredientes,ingrediente,cantidad,unidad
95,Pudding de chía con arándanos y nueces,250 ml leche de almendra,ml leche de almendra,250,unidad


In [17]:
# cambiamos de forma manual

df_ingredientes.loc[95, "ingrediente"] = "leche de almendra"
df_ingredientes.loc[95, "unidad"] = "mililitros"

In [18]:
df_ingredientes.loc[95]

nombre          Pudding de chía con arándanos y nueces
ingredientes                  250 ml leche de almendra
ingrediente                          leche de almendra
cantidad                                           250
unidad                                      mililitros
Name: 95, dtype: object

#### Cantidad 1/2 y 1/4 a 0.5 y 0.25 (Limpieza)

In [19]:
# Indentificar cuantos tenemos

df_ingredientes[(df_ingredientes["cantidad"] == "1/2") |
               (df_ingredientes["cantidad"] == "1/4") |
               (df_ingredientes["cantidad"] == "½") |
               (df_ingredientes["cantidad"] == "¼")
               ]


,nombre,ingredientes,ingrediente,cantidad,unidad
6,"Ensalada marinera: con aguacate, gambas, bacal...",½ Aguacate,Aguacate,½,NaN
7,"Ensalada marinera: con aguacate, gambas, bacal...",½ Limón,Limón,½,NaN
14,Estofado de ternera con verduras,1/2 cebolla,cebolla,1/2,NaN
18,Estofado de ternera con verduras,½ cucharadita de tomín,cucharadita de tomín,½,NaN
34,Hamburguesa con ensalada,¼ de cucharadita de ajo en polvo o ½ ajo fresco,ajo en polvo o ½ ajo fresco,1/4,cucharadita
35,Hamburguesa con ensalada,¼ de cucharadita de cebolla en polvo,cebolla en polvo,1/4,cucharadita
36,Hamburguesa con ensalada,¼ de cucharadita de mostaza Dijon,mostaza Dijon,1/4,cucharadita
75,Piteras de pollo con champiñones y salsa crema,1/4 de cebolla,cebolla,1/4,NaN
79,Piteras de pollo con champiñones y salsa crema,¼ de taza (60 ml) de caldo de pollo,taza (60 ml) de caldo de pollo,¼,NaN
80,Piteras de pollo con champiñones y salsa crema,¼ de taza (60 ml) de crema de leche (nata),taza (60 ml) de crema de leche (nata),¼,NaN


In [20]:
for idx, row in df_ingredientes.iterrows():
    if row["cantidad"] == "½" or row["cantidad"] =="1/2":
        df_ingredientes.at[idx, "ingrediente"] = "1/2" + " " + row["ingrediente"]
        df_ingredientes.at[idx, "cantidad"] = None
        df_ingredientes.at[idx, "unidad"] = None
    
    if row["cantidad"] == "¼" or row["cantidad"] == "1/4":
        df_ingredientes.at[idx, "ingrediente"] = "1/4" + " " + row["ingrediente"]
        df_ingredientes.at[idx, "cantidad"] = None
        df_ingredientes.at[idx, "unidad"] = None
        

### CANTIDAD A NUMÉRICO

In [21]:
df_ingredientes["cantidad"] = pd.to_numeric(df_ingredientes["cantidad"])
df_ingredientes["cantidad"] = df_ingredientes["cantidad"].astype("Int64")


In [22]:
df_ingredientes.dtypes

nombre          object
ingredientes    object
ingrediente     object
cantidad         Int64
unidad          object
dtype: object

### NULOS A VALORES A INTERPRETAR

In [23]:
df_ingredientes.isna().sum()


nombre           0
ingredientes     0
ingrediente      0
cantidad        35
unidad          35
dtype: int64

In [24]:
df_ingredientes["cantidad"] = df_ingredientes["cantidad"].fillna(1)
df_ingredientes["unidad"] = df_ingredientes["unidad"].fillna("unidad")

In [25]:
df_ingredientes.isna().sum()

nombre          0
ingredientes    0
ingrediente     0
cantidad        0
unidad          0
dtype: int64

### PREPARADO FINAL Y GUARDADO DE CSV: recetas_ingredientes_cantidades

In [26]:
# Seleccionamos solo las columnas finales

df_ing_cant = df_ingredientes.drop(columns={"ingredientes"})

In [27]:
df_ing_cant

,nombre,ingrediente,cantidad,unidad
0,"Ensalada marinera: con aguacate, gambas, bacal...",Aceite de oliva,10,gramos
1,"Ensalada marinera: con aguacate, gambas, bacal...",Gambas,100,gramos
2,"Ensalada marinera: con aguacate, gambas, bacal...",bacalao ahumado,100,gramos
3,"Ensalada marinera: con aguacate, gambas, bacal...",nueces,20,gramos
4,"Ensalada marinera: con aguacate, gambas, bacal...","Hojas verdes (lechuga, espinacas, rúcula, kale…)",70,gramos
...,...,...,...,...
124,Salmón a la plancha con salsa y brócoli,Sal y pimienta al gusto (salsa limón),1,unidad
125,"Yogur griego con fresas, almendras y nueces.",fraulas,150,gramos
126,"Yogur griego con fresas, almendras y nueces.",almendras,20,gramos
127,"Yogur griego con fresas, almendras y nueces.",nueces,20,gramos


In [28]:
# Estandarizamos el texto en "ingredientes". Primera en mayus

df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].str.lower()

In [29]:
# Eliminamos espacios a ambos lados de los valores string en variables

columnas_string = ['ingrediente', 'nombre', 'unidad']  
for columna in columnas_string:
    df_ing_cant[columna] = df_ing_cant[columna].str.strip()

In [30]:
# Observamos algunos valores en "ingrediente" que necesitan estandarización

df_ing_cant[df_ing_cant["ingrediente"].str.contains("sal ")]

,nombre,ingrediente,cantidad,unidad
5,"Ensalada marinera: con aguacate, gambas, bacal...",sal y pimienta,1,unidad
17,Estofado de ternera con verduras,sal y pimienta al gusto,1,unidad
28,Fumet de gambas (ceto),sal al gusto,1,unidad
33,Hamburguesa con ensalada,sal y pimienta al gusto,1,unidad
49,Merluza con corteza de almendras y parmesano +...,sal y pimienta al gusto,1,unidad
69,Pastel de pescado,sal y pimienta,1,unidad
78,Piteras de pollo con champiñones y salsa crema,sal y pimienta al gusto,1,unidad
103,Revuelto de espárragos y feta,sal y pimienta buena,1,unidad
114,Rollitos de col,sal y pimienta al gusto,1,unidad
123,Salmón a la plancha con salsa y brócoli,sal y pimienta al gusto,1,unidad


In [31]:
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("sal y pimienta al gusto", "sal y pimienta")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("sal y pimienta buena", "sal y pimienta")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("sal al gusto", "sal")

In [32]:
# visualización del conteo de ingredientes para afinar la estandarización

print(f"Valores únicos en ingredientes: {df_ing_cant['ingrediente'].nunique()}")

print(df_ing_cant['ingrediente'].value_counts().sort_values(ascending=False).head(50))
print(df_ing_cant['ingrediente'].value_counts().sort_values(ascending=False).tail(50))

Valores únicos en ingredientes: 100
ingrediente
sal y pimienta                                      9
ajo                                                 5
aceite de oliva                                     5
nueces                                              3
huevos                                              3
huevo                                               3
queso mozzarella                                    2
mantequilla                                         2
1/2 cebolla                                         2
coliflor                                            2
sal                                                 2
queso feta                                          2
leche de almendra                                   2
carne de ternera para estofar, troceada             1
hojas verdes (lechuga, espinacas, rúcula, kale…)    1
1/2 aguacate                                        1
1/2 limón                                           1
tomate concentrado (opcional)     

In [33]:
# Cambios de estandarización al observar conteo

df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("1⁄2 limón", "1/2 limón")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("1/4 ajo en polvo o ½ ajo fresco", "1/4 ajo en polvo o 1/2 ajo fresco")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("aceite de oliva (aove)", "aceite de oliva")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("huevos", "huevo")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("aceite de oliva (10g)", "aceite de oliva")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("carne de ternera para estofar, troceada", "carne de ternera")
df_ing_cant["ingrediente"] = df_ing_cant["ingrediente"].replace("carne de ternera del estofado del otro día", "carne de ternera")

df_ing_cant["nombre"] = df_ing_cant["nombre"].replace("Ensalada marinera: con aguacate, gambas, bacalao ahumado y", "Ensalada marinera: con aguacate, gambas, bacalao ahumado y nueces")

In [34]:
# cambiamos cantidad y unidad de "aceite de oliva"
# probando cambios por vectores en lugar de ITERROWS, es mas rápido

df_ing_cant.loc[df_ing_cant["ingrediente"] == "aceite de oliva", ["cantidad", "unidad"]] = [1, "cucharada"]

In [44]:
# cambiamos "sal" y "sal y pimienta"

df_ing_cant.loc[df_ing_cant["ingrediente"] == "sal", ["cantidad", "unidad"]] = [None, None]
df_ing_cant.loc[df_ing_cant["ingrediente"] == "sal y pimienta", ["cantidad", "unidad"]] = [None, None]

In [35]:
# Vistazo al n de recetas y cuales son

print(f"Recetas:")
print(f"\n{df_ing_cant['nombre'].unique()}")
print(f"\n N total de recetas: {df_ing_cant['nombre'].nunique()}")

Recetas:

['Ensalada marinera: con aguacate, gambas, bacalao ahumado y nueces'
 'Estofado de ternera con verduras' 'Fumet de gambas (ceto)'
 'Hamburguesa con ensalada'
 'Huevos revueltos con de salmón, espinacas y queso crema y feta'
 'Merluza con corteza de almendras y parmesano + ensalada verde'
 'Pastel de chocolate' 'Pastel de pescado'
 'Piteras de pollo con champiñones y salsa crema'
 'Pizza con base de coliflor' 'Pudding de chía con arándanos y nueces'
 'Revuelto de espárragos y feta' 'Rollitos de col'
 'Salmón a la plancha con salsa y brócoli'
 'Yogur griego con fresas, almendras y nueces.']

 N total de recetas: 15


In [46]:
df_ing_cant.to_csv("recetas_ingredientes_cantidades.csv", index=False)

### BORRADOR/ CAMPO PRUEBAS

In [37]:
df_ing_cant.groupby(["ingrediente", "unidad"])["cantidad"].sum().reset_index().head(10)

,ingrediente,unidad,cantidad
0,0.5g de sal,unidad,1
1,1/2 aguacate,unidad,1
2,1/2 cebolla,unidad,2
3,1/2 cucharadita de tomín,unidad,1
4,1/2 limón,unidad,2
5,1/4 ajo en polvo o 1/2 ajo fresco,unidad,1
6,1/4 cebolla,unidad,1
7,1/4 cebolla en polvo,unidad,1
8,1/4 mostaza dijon,unidad,1
9,1/4 taza (60 ml) de caldo de pollo,unidad,1


In [50]:
df_ing_cant[df_ing_cant["ingrediente"].str.contains("salsa limón")]

,nombre,ingrediente,cantidad,unidad
116,Salmón a la plancha con salsa y brócoli,"ajo, picado (salsa limón)",1,unidad
118,Salmón a la plancha con salsa y brócoli,mantequilla (salsa limón),10,gramos
120,Salmón a la plancha con salsa y brócoli,zumo de limón (salsa limón),2,cucharada
121,Salmón a la plancha con salsa y brócoli,perejil fresco picado (salsa limón),1,unidad
122,Salmón a la plancha con salsa y brócoli,ralladura de un limón (salsa limón),1,unidad
124,Salmón a la plancha con salsa y brócoli,sal y pimienta al gusto (salsa limón),1,unidad


In [47]:
df_ing_cant[df_ing_cant["ingrediente"].str.contains("sal")]

,nombre,ingrediente,cantidad,unidad
5,"Ensalada marinera: con aguacate, gambas, bacal...",sal y pimienta,<NA>,None
17,Estofado de ternera con verduras,sal y pimienta,<NA>,None
28,Fumet de gambas (ceto),sal,<NA>,None
30,Hamburguesa con ensalada,ensalada verde,100,gramos
33,Hamburguesa con ensalada,sal y pimienta,<NA>,None
38,"Huevos revueltos con de salmón, espinacas y qu...",salmón ahumado,100,gramos
48,Merluza con corteza de almendras y parmesano +...,mantequilla sin sal,25,gramos
49,Merluza con corteza de almendras y parmesano +...,sal y pimienta,<NA>,None
50,Pastel de chocolate,0.5g de sal,1,unidad
66,Pastel de pescado,ml salsa de tomate (casera o sin aditivos),250,unidad
